In [2]:
%matplotlib inline
import numpy as np
import random
import time
from keras.models import load_model
from scipy import fftpack
import os, sys
import scipy.ndimage as ndimage
import healpy as hp
from astropy.io import fits
from importlib import reload 
from scipy.stats import pearsonr
import scipy.stats as stats
from scipy import interpolate
import scipy.ndimage as ndimage

import gc

import matplotlib.pyplot as plt

# sys path prepend and reload are for development
sys.path = ['../'] + sys.path
print(sys.path)

['../', '../', '', '/Users/ucapnje/anaconda3/lib/python36.zip', '/Users/ucapnje/anaconda3/lib/python3.6', '/Users/ucapnje/anaconda3/lib/python3.6/lib-dynload', '/Users/ucapnje/.local/lib/python3.6/site-packages', '/Users/ucapnje/anaconda3/lib/python3.6/site-packages', '/Users/ucapnje/anaconda3/lib/python3.6/site-packages/aeosa', '/Users/ucapnje/anaconda3/lib/python3.6/site-packages/pydelfi-0.1-py3.6.egg', '/Users/ucapnje/anaconda3/lib/python3.6/site-packages/deepmass-0.1-py3.6.egg', '/Users/ucapnje/anaconda3/lib/python3.6/site-packages/skymapper-0.3.2-py3.6.egg', '/Users/ucapnje/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/ucapnje/.ipython']


In [3]:
# sys path prepend and reload are for development

from deepmass import map_functions as mf
from deepmass import lens_data as ld
from deepmass import wiener

reload(mf)
reload(ld)
reload(wiener)

<module 'deepmass.wiener' from '../deepmass/wiener.py'>

In [4]:
sv_mask = np.load('sv_mask.npy')

In [5]:
# trained_model_file = '../outputs/h5_files/model_unet_weights_simplest3e-06.h5'
trained_model_file = '../outputs/h5_files/model_unet_simplest1e-05.h5'
model = load_model(trained_model_file)

In [6]:
wiener_array = np.load('../picola_training/validation_data/test_array_wiener.npy')
gc.collect()

7

In [7]:
clean_array = np.load('../picola_training/validation_data/test_array_clean.npy')
gc.collect()

3

In [8]:
flattened_masked_wiener = [(ndimage.gaussian_filter(wiener_array[i,:,:,0], sigma=0.5).flatten())[np.where(sv_mask.flatten()>0.5)] \
                           for i in range(wiener_array.shape[0])]
flattened_masked_clean = [(ndimage.gaussian_filter(clean_array[i,:,:,0], sigma=0.5).flatten())[np.where(sv_mask.flatten()>0.5)] \
                          for i in range(clean_array.shape[0])]

flattened_masked_wiener = np.array(flattened_masked_wiener).flatten()
flattened_masked_clean = np.array(flattened_masked_clean).flatten()

# Apply model

In [9]:
t=time.time()
output_array = model.predict(wiener_array)
gc.collect()
print(time.time() -t)

3227.8084597587585


In [10]:
print(np.mean((output_array.flatten() - clean_array.flatten())**2.))
print(np.mean((output_array.flatten() - clean_array.flatten())**2.)/9.)


0.00042692028
4.7435586263115205e-05


In [11]:
print(np.mean((wiener_array.flatten() - clean_array.flatten())**2.))
print(np.mean((wiener_array.flatten() - clean_array.flatten())**2.)/9.)


0.0004627225
5.141361099150446e-05


In [13]:
print(np.mean((output_array.flatten() - clean_array.flatten())**2.)/
      np.mean((wiener_array.flatten() - clean_array.flatten())**2.) - 1.)

-0.0773729681968689


In [14]:
flattened_masked_output = [(ndimage.gaussian_filter(output_array[i,:,:,0], sigma=0.5).flatten())[np.where(sv_mask.flatten()>0.5)] \
                           for i in range(output_array.shape[0])]

flattened_masked_output = np.array(flattened_masked_output).flatten()

In [15]:
print(np.mean((flattened_masked_output - flattened_masked_clean)**2.))
print(np.mean((flattened_masked_output - flattened_masked_clean)**2.)/9.)

0.0005752247
6.391385492558281e-05


In [16]:
print(np.mean((flattened_masked_wiener - flattened_masked_clean)**2.))
print(np.mean((flattened_masked_wiener - flattened_masked_clean)**2.)/9.)

0.0006440086
7.15565143360032e-05


In [17]:
print(np.mean((flattened_masked_output - flattened_masked_clean)**2.)/
      np.mean((flattened_masked_wiener - flattened_masked_clean)**2.) - 1.)

-0.10680592060089111
